In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import re
import copy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def url_to_params(url):
  split_url = url.split('?')
  b_url = split_url[0]
  
  querystring = split_url[1].split('&')
  querystring2 = []
  params = {}
  for i in range(len(querystring)):
    querystring2.append(querystring[i].split('='))

  for i in range(len(querystring2)):
    params[querystring2[i][0]] = querystring2[i][1]

  return b_url ,params

In [ ]:
main_url = 'https://www.musinsa.com/app/reviews/lists?type=style&year_date=2022&month_date=1&day_date=&max_rt=2022&min_rt=2009&brand=&page=1&sort=new&hash_id=&best_type=&s_type=all&q='
b_url, params = url_to_params(main_url)

In [ ]:
params

{'best_type': '',
 'brand': '',
 'day_date': '',
 'hash_id': '',
 'max_rt': '2022',
 'min_rt': '2009',
 'month_date': '1',
 'page': '1',
 'q': '',
 's_type': 'all',
 'sort': 'new',
 'type': 'style',
 'year_date': '2022'}

In [ ]:
year_list = [2018, 2019, 2020]

## 무신사 댓글 페이지가 1001페이지를 넘어가면 1페이지로 돌아가짐, 1001페이지 까지만 사용할 수 있으므로 2021년 데이터 추가 크롤링
- 따라서 total page를 뽑는 크롤링 구현은 하였으나 불필요

In [ ]:
# total_pages = []
# for m in tqdm(range(1, 13)):  # range(1, 6)
#     params['month_date'] = str(m)
#     header = {
#         'referer': 'https://www.musinsa.com/app/reviews/lists?type=style',
#         'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.108 Whale/3.15.136.18 Safari/537.36'
#     }

#     resp = requests.get(url=b_url, params=params, headers=header)
#     soup = BeautifulSoup(resp.text, 'lxml')
#     page_tags = soup.select('div.box_page_msg')

#     total_page = int(page_tags[0].text.strip().split(' ')[0].replace(',', ''))
#     total_pages.append(total_page)

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
# total_pages

[3884, 3666, 4709, 4434, 4269, 5123, 5124, 5040, 5896, 9080, 10243, 9908]

## 아래 실제 하용할 크롤링 코드에서 total_pages 리스트가 필요함,
- 1001 페이지 까지만 크롤링 하기 때문에 1001로만 저장된 리스트 생성

In [ ]:
total_pages = [1001] * 12
total_pages

[1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001]

## 년, 월에 맞춰 파라미터를 변경하여 리스트에 저장

In [ ]:
params_list = []
for y in year_list:
    params['year_date'] = str(y)
    for p in range(1, 13): # range(1, 6)
        params['month_date'] = str(p)
        make_parmas = copy.deepcopy(params)
        params_list.append(make_parmas)

In [20]:
params_list

[{'best_type': '',
  'brand': '',
  'day_date': '',
  'hash_id': '',
  'max_rt': '2022',
  'min_rt': '2009',
  'month_date': '1',
  'page': '7',
  'q': '',
  's_type': 'all',
  'sort': 'new',
  'type': 'style',
  'year_date': '2018'},
 {'best_type': '',
  'brand': '',
  'day_date': '',
  'hash_id': '',
  'max_rt': '2022',
  'min_rt': '2009',
  'month_date': '2',
  'page': '1',
  'q': '',
  's_type': 'all',
  'sort': 'new',
  'type': 'style',
  'year_date': '2018'},
 {'best_type': '',
  'brand': '',
  'day_date': '',
  'hash_id': '',
  'max_rt': '2022',
  'min_rt': '2009',
  'month_date': '3',
  'page': '1',
  'q': '',
  's_type': 'all',
  'sort': 'new',
  'type': 'style',
  'year_date': '2018'},
 {'best_type': '',
  'brand': '',
  'day_date': '',
  'hash_id': '',
  'max_rt': '2022',
  'min_rt': '2009',
  'month_date': '4',
  'page': '1',
  'q': '',
  's_type': 'all',
  'sort': 'new',
  'type': 'style',
  'year_date': '2018'},
 {'best_type': '',
  'brand': '',
  'day_date': '',
  'hash_

## 필요 정보 크롤링하여 com_dic에 저장

In [21]:
header = {
    'referer': 'https://www.musinsa.com/app/reviews/lists?type=style',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.108 Whale/3.15.136.18 Safari/537.36'
}

In [22]:
com_dic = {
    'month' : [],
    'page' : [],
    'comment' : [],
    'star' : []
}

for tp, pl in zip(total_pages, params_list):
    for i in tqdm(range(1, tp + 1)):
        pl['page'] = str(i)

        resp = requests.get(url=b_url, params=pl, headers=header)
        soup = BeautifulSoup(resp.text, 'lxml')

        com_tags = soup.select('div.review-contents__text')
        star_tags = soup.select('div.review-list__rating-wrap > span > span > span')

        for com, star in zip(com_tags, star_tags):
            if int(int(star['style'][7:-1]) / 20) == 5:
                pass
            else:
                com_dic['month'].append(pl['month_date'])
                com_dic['page'].append(pl['page'])
                com_dic['comment'].append(com.text.strip().replace('\n', ' '))
                com_dic['star'].append(int(int(star['style'][7:-1]) / 20))

  0%|          | 0/1001 [00:00<?, ?it/s]

  0%|          | 0/1001 [00:00<?, ?it/s]

  0%|          | 0/1001 [00:00<?, ?it/s]

  0%|          | 0/1001 [00:00<?, ?it/s]

  0%|          | 0/1001 [00:00<?, ?it/s]

...

  0%|          | 0/1001 [00:00<?, ?it/s]

  0%|          | 0/1001 [00:00<?, ?it/s]

  0%|          | 0/1001 [00:00<?, ?it/s]

  0%|          | 0/1001 [00:00<?, ?it/s]

  0%|          | 0/1001 [00:00<?, ?it/s]

In [23]:
com_dic

{'comment': ['사이즈도 맞고 사진이랑 색도 같네요  옷도 이쁘고 마음에 들어요 마감이 살짝 아쉽긴 한데 대체적으로 만족합니다 ㅎㅎ',
  '너무 딱 맞아서 사이즈 바꿀려다가 입으면 늘어날 재질같아서 그냥 입기로했어요 도톰하고 요즘날씨에 입어도 안춥네요',
  '기장이 많이 길진 않구요. 엉덩이 가릴정도로 오버핏은 아니고 모자도 그렇게 크진않아요 평범함에 약간 오버핏을 더한느낌 근데 재질자체는 보들보들한게 느낌이 좋고 가슴넓이나 어깨 팔 핏은 느낌 좋아요. 길이감 좀더 길고 밴딩이 약간 덜 짱짱햇다면 완전 취향이엇을듯싶어요. 원래 엄청 강한 오버핏 느낌의 후드를 좋아하는편이라서 쏘쏘함',
  '제가 올린 사진이 거지같아서 그렇지 실물이 더 예쁘긴 한데 색이 너무 바랜 원단이라 개인적으로는 아쉽습니다ㅜ',
  '그렇게 레쟈느낌이 강하지는 않고 핏도 무난 무난합니다. 그런데 사진보다는 버튼이 좀 더 눈에 들어오는 편입니다.',
  '촉감도 그렇게 나쁘지 않고 어깨라인이 떨어지는 오버사이즈 느낌인데 팔은 또 길지가 않네요.',
  '약간 어깨깡패같은 느낌이 됩니다. 어깨 라인은 떨어지는 게 레귤러 핏은 아니고 팔은 또 짧습니다. 재질은 무난합니다.',
  '저렴하게 조거팬츠 잘 샀네요 검정츄리닝 바지 필요했는데 좋아요 다만 배송이 오배송되어 고객센터와 통화하고 트러블이 발생함',
  '가격대비 좋고. 부들부들하니 오버핏으로 축 늘어뜨려서 입어요 요즘 날씨에 적합하고. 이너도 좋습니다',
  '161 95착용 무릎밑정도오고 무거운편이고 단점이 더 많지만 무난무난하게 잘 입고있습니다 소매에 비밀주머니 있는데 이게젤맘에듬ㅋ_ㅋ',
  '177cm에 78kg 입니다 벨트 라인에 딱 떨어져서 마음에 듭니다 팔 길이가 다른 사람들보다 긴 편인데 소매길이가 딱 맞습니다 가격과 품질 만족스럽습니다 :)',
  '몸에 잘맞습니다 치수거의정확하구요 잘입겠습니다 싸게 잘구입했습니다 무신사 감사함니다',
  '어깨가 좀 넓은 감이 있는데 안에 좀 껴 입으면 무난하게 입을수있네요

## 추출한 데이터 DataFrame으로 변경 및 확인, 중복데이터 확인하여 Drop 후 csv파일로 저장

In [24]:
df = pd.DataFrame(com_dic)
# df.to_csv(path + 'comment_data_mouth4.csv')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25713 entries, 0 to 25712
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   month    25713 non-null  object
 1   page     25713 non-null  object
 2   comment  25713 non-null  object
 3   star     25713 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 803.7+ KB


In [26]:
df.nunique()

month         12
page        1001
comment    21136
star           4
dtype: int64

In [27]:
df.drop_duplicates(['comment'], inplace=True) # 중복 데이터를 drop시키는 함수
# df['comment'].duplicated() # duplicated : 중복 데이터 여부를 True / False로 보여줌
df

,month,page,comment,star
0,1,1,사이즈도 맞고 사진이랑 색도 같네요 옷도 이쁘고 마음에 들어요 마감이 살짝 아쉽긴...,4
1,1,2,너무 딱 맞아서 사이즈 바꿀려다가 입으면 늘어날 재질같아서 그냥 입기로했어요 도톰하...,4
2,1,2,기장이 많이 길진 않구요. 엉덩이 가릴정도로 오버핏은 아니고 모자도 그렇게 크진않아...,4
3,1,2,제가 올린 사진이 거지같아서 그렇지 실물이 더 예쁘긴 한데 색이 너무 바랜 원단이라...,4
4,1,4,그렇게 레쟈느낌이 강하지는 않고 핏도 무난 무난합니다. 그런데 사진보다는 버튼이 좀...,4
...,...,...,...,...
25708,12,1000,10만원 코트치고는 이쁘고 좋은 것 같네요. 한 가지 아쉬운점은 내부주머니가 바느질...,4
25709,12,1001,길이 두깨겜 전부 만족! 여름용으로도 만둘어쥬세요 또 살거니까요!,4
25710,12,1001,생각보다 너무 박시해서 당황했지만 옷이 넘기엽구 안이 기모라 넘넘 따뜻하네영!,4
25711,12,1001,부드러울거 생각했는데 그렇게 부드럽진않네요ㅠ 근데 괜찮아요 색깔이 원하던 색깔이라 ...,3


In [28]:
path = '/content/drive/MyDrive/Colab Notebooks/alpaco/Team_1. CODEnter [ Save Form : Name_FileName_Date ]/2. SUB _ MUSINSA/4. csv/'

In [29]:
df.to_csv(path + 'comment_data_add_neg.csv')